In [ ]:
import scipy.io
import numpy as np
from sklearn.preprocessing import scale, LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Dense

In [3]:
fname = "data/train_4.mat"
train_mat = scipy.io.loadmat(fname)
fname = "data/test_4.mat"
test_mat = scipy.io.loadmat(fname)

dataset = train_mat.get("feat") # or use emg_data_train for using all 16 gestures
trainX = []
trainY = []
for i, entry in np.ndenumerate(dataset):
    # i is a tuple of (row, column)
    # save readings into data array
    trainX.append(dataset[i[0],i[1]])
    # gesture class is put into labels array
    trainY.append(i[1])

# similarly do for the test data
dataset = test_mat.get("feat") # or use emg_data_test for using all 16 gestures
testX = []
testY = []
for i, entry in np.ndenumerate(dataset):
    testX.append(dataset[i[0],i[1]])
    testY.append(i[1])


In [4]:
# convert the labels from integers/categories to vectors 
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)   # fit_transform = find all unique class labels + transform into one-hot encoded labels
testY = lb.transform(testY)         # transform = perform the one-hot encoding (unique class labels already found)

In [5]:
lb.classes_

array([0, 1, 2, 3])

Define neural network architecture using Keras

In [ ]:
model = Sequential()   # input layer 3072 as there are 32x32x3=3072 pixels in a flattened input image
model.add(Dense(1024, input_shape=(3072,), activation="relu"))   
model.add(Dense(512, activation="relu"))                         
model.add(Dense(len(lb.classes_), activation="softmax"))    